In [ ]:

import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam


In [ ]:
import tensorflow as tf

# Verificar si hay GPU disponible
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Activar uso de memoria dinámico
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPU detectada: {gpus[0].name}")
    except RuntimeError as e:
        print(f"Error al configurar GPU: {e}")
else:
    print("No se detectó GPU. Usando CPU.")


In [ ]:

def load_images_from_folder(folder, img_size=(512, 512)):
    images = []
    for filename in os.listdir(folder):
        path = os.path.join(folder, filename)
        img = load_img(path, target_size=img_size)
        img = img_to_array(img) / 255.0  # normalizar
        images.append(img)
    return np.array(images)


In [ ]:

base_path = 'c:/Users/qdari/OneDrive/Documentos/aprendizajeSupervisado/planttraits2024'  ###
X = load_images_from_folder(os.path.join(base_path, 'images_original'))
Y = load_images_from_folder(os.path.join(base_path, 'images_clean'))
X_test = load_images_from_folder(os.path.join(base_path, 'images_test'))

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.1, random_state=42)

print(f'Train shape: {X_train.shape}, Validation shape: {X_val.shape}')


In [ ]:

input_img = Input(shape=(512, 512, 3))  # Nueva entrada

# ===== Codificador =====
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)      # 512x512x32
x = MaxPooling2D((2, 2), padding='same')(x)                                # 256x256x32

x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)              # 256x256x64
x = MaxPooling2D((2, 2), padding='same')(x)                                # 128x128x64

x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)             # 128x128x128
x = MaxPooling2D((2, 2), padding='same')(x)                                # 64x64x128

x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)             # 64x64x256
x = MaxPooling2D((2, 2), padding='same')(x)                                # 32x32x256

x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)             # 32x32x512
x = MaxPooling2D((2, 2), padding='same')(x)                                # 16x16x512

x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)             # 16x16x512
encoded = MaxPooling2D((2, 2), padding='same')(x)                          # 8x16x512

# ===== Decodificador =====
x = Conv2D(512, (3, 3), activation='relu', padding='same')(encoded)       # 8x8x512
x = UpSampling2D((2, 2))(x)                                                # 16x16x512

x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)             # 16x16x256
x = UpSampling2D((2, 2))(x)                                                # 32x32x256

x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)             # 32x32x128
x = UpSampling2D((2, 2))(x)                                                # 64x64x128

x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)              # 64x64x64
x = UpSampling2D((2, 2))(x)                                                # 128x128x64

x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)             # 128x128x32
x = UpSampling2D((2, 2))(x)                                                # 256x256x32

x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)              # 256x256x32
x = UpSampling2D((2, 2))(x)                                                # 512x512x32

# Capa de salida
decoded = Conv2D(3, (3, 3), activation='relu', padding='same')(x)      # 512x512x3

# Modelo autoencoder
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer=Adam(), loss='mse')
autoencoder.summary()

In [ ]:
augmenter = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# X_train: imágenes ruidosas, Y_train: imágenes limpias
batch_size = 32

# Crear generadores sincronizados para (X, Y)
train_generator = augmenter.flow(X_train, batch_size=batch_size, seed=42)
target_generator = augmenter.flow(Y_train, batch_size=batch_size, seed=42)

# Generador combinado: entrega pares (X_aug, Y_aug)
def combined_generator(gen1, gen2):
    while True:
        yield next(gen1), next(gen2)

augmented_data = combined_generator(train_generator, target_generator)

# Entrenamiento con imágenes aumentadas
autoencoder.fit(
    augmented_data,
    steps_per_epoch=len(X_train) // batch_size,
    validation_data=(X_val, Y_val),
    epochs=30,
    callbacks=[early_stop]
)

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = autoencoder.fit(X_train, Y_train,
                          epochs=50,
                          batch_size=32,
                          shuffle=True,
                          validation_data=(X_val, Y_val),
                          callbacks=[early_stop])


In [ ]:

start_index = 135           

X_test = load_images_from_folder(os.path.join(base_path, 'images_test'), img_size=(512, 512))
X_denoised = autoencoder.predict(X_test)

# Mostrar comparativa: ruidosa vs reconstruida
n = 5  # número de imágenes a mostrar
plt.figure(figsize=(15, 6))
for i in range(n):
    idx = start_index + i
    if idx >= len(X_test):
        break
    ax = plt.subplot(2, n, i + 1)  # Imagen ruidosa
    plt.imshow(X_test[idx])
    plt.title(f"Ruidosa {idx}")
    plt.axis("off")
    ax = plt.subplot(2, n, i + 1 + n)  # Imagen denoised
    plt.imshow(X_denoised[idx])
    plt.title(f"Denoised {idx}")
    plt.axis("off")

plt.tight_layout()
plt.show()



In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

Y_test = load_images_from_folder(os.path.join(base_path, 'images_clean'), img_size=(512, 512))
Y_denoised = autoencoder.predict(Y_test)

mse_values = [mean_squared_error(x_true.flatten(), x_pred.flatten())
              for x_true, x_pred in zip(Y_test, Y_denoised)]

print(f"MSE promedio en prueba: {np.mean(mse_values):.4f}")

In [ ]:
import cv2

def psnr(img1, img2):
    return cv2.PSNR((img1 * 255).astype(np.uint8), (img2 * 255).astype(np.uint8))

psnr_values = [psnr(x_true, x_pred) for x_true, x_pred in zip(Y_test, Y_denoised)]
print(f"PSNR promedio: {np.mean(psnr_values):.2f} dB")